In [1]:
from qiskit_library import *
import torch
import numpy as np
from random import randrange
import qiskit as qk
from qiskit import Aer
from qiskit import execute
import math
import sys
import random
import time
from tqdm import tqdm

from qiskit import IBMQ
# IBMQ.delete_accounts()
IBMQ.save_account('62d0e14364f490e45b5b5e0f6eebdbc083270ffffb660c7054219b15c7ce99ab4aa3b321309c0a9d0c3fc20086baece1376297dcdb67c7b715f9de1e4fa79efb')
IBMQ.load_account()




def analyze(counts):
    mycount = {}
    for i in range(num_c_reg):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits



def fire_ibmq(circuit,shots,iter,Simulation = False, printable=True,backend_name='ibmq_essex'):
    if printable:
        print(circuit)
    
    count_set = []
    start = time.time()
    for it in range(iter):
        if not Simulation:
            provider = IBMQ.get_provider('ibm-q-academic')
            # ibm-q-academic backends: 
            #  5 qubits: ibmq_valencia
            # 20 qubits: ibmq_poughkeepsie, ibmq_johannesburg,ibmq_boeblingen, ibmq_20_tokyo
            # 53 qubits: ibmq_rochester
            
            # To get a specific qubit backend: 
            backend = provider.get_backend(backend_name)
        else:
            backend = Aer.get_backend('qasm_simulator')
        job_ibm_q = execute(circuit, backend, shots=shots)
        job_monitor(job_ibm_q)
        result_ibm_q = job_ibm_q.result()
        counts = result_ibm_q.get_counts()
        count_set.append(counts)
    end = time.time()
    print("Simulation time:", end - start)

    return count_set


def do_slp_via_th(input_ori,w_ori):
    p = input_ori
    d = 4*p*(1-p)
    e = (2*p-1)
    # e_sq = torch.tensor(1)
    w = w_ori
    
    sum_of_sq = (d+e.pow(2)).sum(-1)
    sum_of_sq = sum_of_sq.unsqueeze(-1)        
    sum_of_sq = sum_of_sq.expand(p.shape[0], w.shape[0])
            
    diag_p = torch.diag_embed(p)        
    
    p_w = torch.matmul(w,diag_p)
    
    z_p_w = torch.zeros_like(p_w)        
    shft_p_w = torch.cat((p_w, z_p_w), -1)
    
    sum_of_cross = torch.zeros_like(p_w)
    # print(p,p.shape)
    length = p.shape[1]    
    
    for shft in range(1,length):    
        sum_of_cross += shft_p_w[:,:,0:length]*shft_p_w[:,:,shft:length+shft]

    sum_of_cross = sum_of_cross.sum(-1)
            
    return (sum_of_sq+2*sum_of_cross)/(length**2) 



/home/weiwen/.local/lib/python3.6/site-packages/qiskit/providers/ibmq/credentials/configrc.py:130: UserWarning: Credentials already present. Set overwrite=True to overwrite.
  warnings.warn('Credentials already present. '


RequestsApiError: HTTPSConnectionPool(host='auth.quantum-computing.ibm.com', port=443): Max retries exceeded with url: /api/version (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fc39b1b7780>: Failed to establish a new connection: [Errno -2] Name or service not known',))

In [95]:
# 2-1 ENCODER + Weights
input_ori = [0.9,0.1]
input = torch.tensor(input_ori)*2-1
w1    = [ 1.0, 1.0]
w5    = [ 1.0, 0.0]

q_io = qk.QuantumRegister(2,"io")
q_n0 = qk.QuantumRegister(1,"neural_0")
q_n1 = qk.QuantumRegister(1,"neural_1")
c = qk.ClassicalRegister(2,"reg")

circuit = qk.QuantumCircuit(q_io, q_n0, q_n1, c)


SLP_2_encoding(circuit,q_io,q_n0,input)
SLP_2_Uw(circuit,q_n0,w1)
# circuit.barrier()

circuit.h(q_n1)
circuit.cz(q_io[1],q_n1)
SLP_2_Uw(circuit,q_n1,w5)
circuit.barrier()

circuit.h(q_n0)
circuit.h(q_n1)
circuit.x(q_n0)
circuit.x(q_n1)

circuit.barrier()
circuit.measure(q_n0,c[0])
circuit.measure(q_n1,c[1])
print(circuit)



               ┌────────────┐         ░               ░            ░       
      io_0: |0>┤ Ry(0.6435) ├──■──────░───────────────░────────────░───────
               ├────────────┤┌─┴─┐    ░               ░            ░       
      io_1: |0>┤ Ry(2.4981) ├┤ X ├─■──░───────■───────░────────────░───────
               └───┬───┬────┘└───┘ │  ░       │       ░ ┌───┐┌───┐ ░ ┌─┐   
neural_0_0: |0>────┤ H ├───────────■──░───────┼───────░─┤ H ├┤ X ├─░─┤M├───
                   └───┘              ░ ┌───┐ │ ┌───┐ ░ ├───┤├───┤ ░ └╥┘┌─┐
neural_1_0: |0>───────────────────────░─┤ H ├─■─┤ Z ├─░─┤ H ├┤ X ├─░──╫─┤M├
                                      ░ └───┘   └───┘ ░ └───┘└───┘ ░  ║ └╥┘
      reg_0: 0 ═══════════════════════════════════════════════════════╩══╬═
                                                                         ║ 
      reg_1: 0 ══════════════════════════════════════════════════════════╩═
                                                                           


In [ ]:
# 2-1 ENCODER + Weights + 2 layers
input_ori = [0.1,0.9]
input = torch.tensor(input_ori)*2-1
w1    = [ 1.0, 1.0]
w5    = [ 1.0, 0.0]

q_io = qk.QuantumRegister(2,"io")
q_n0 = qk.QuantumRegister(1,"neural_0")
q_n1 = qk.QuantumRegister(1,"neural_1")
q_n2 = qk.QuantumRegister(1,"neural_2")
c = qk.ClassicalRegister(1,"reg")

circuit = qk.QuantumCircuit(q_io, q_n0, q_n1, q_n2, c)


SLP_2_encoding(circuit,q_io,q_n0,input)
SLP_2_Uw(circuit,q_n0,w1)
# circuit.barrier()

circuit.h(q_n1)
circuit.cz(q_io[1],q_n1)
SLP_2_Uw(circuit,q_n1,w5)
circuit.barrier()

circuit.h(q_n0)
circuit.h(q_n1)
circuit.x(q_n0)
circuit.x(q_n1)
circuit.barrier()

SLP_2_encoding(circuit,[q_n0,q_n1],q_n2)
SLP_2_Uw(circuit,q_n2,w5)

circuit.h(q_n2)
circuit.x(q_n2)

circuit.measure(q_n2,c)
# circuit.measure(q_n1,c[1])
print(circuit)



In [ ]:
input_ori = [0.319,0.869,0.061,0.995]
input = 1-torch.tensor(input_ori)*2
w5    = [ 1.0, 1.0, 1.0, 1.0]

q_io = qk.QuantumRegister(4,"io")
q_en = qk.QuantumRegister(2,"encoded")
q_aux = qk.QuantumRegister(1,"aux")
q_out = qk.QuantumRegister(1,"out")
c = qk.ClassicalRegister(1,"reg")

circuit = qk.QuantumCircuit(q_io, q_en, q_aux, q_out, c)

SLP_4_encoding(circuit,q_io,q_en,input,q_aux)
# reset_qbits(circuit,q_io)
SLP_4_Uw(circuit,q_en,w5,q_aux)
circuit.barrier()
circuit.h(q_en)
circuit.x(q_en)

circuit.ccx(q_en[0],q_en[1],q_out)
circuit.measure(q_out,c)
print(circuit)


# 
# print("="*50)
# print("Start theoretic:")
# start = time.time() 
# print(do_slp_via_th(t_input,t_w1))


In [ ]:
qc_shots = 1024
num_c_reg = 4

t_inter = do_slp_via_th(t_input,t_w1)*2-1 
t_w2 = torch.tensor([w5])*2-1
t_output = do_slp_via_th(t_inter,t_w2) 
print(t_output)
end = time.time()
qc_time = end - start
print("Theoretic elasped time:",qc_time)

In [ ]:
print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = fire_ibmq(circuit,qc_shots,iters,True,False)
end = time.time()
qc_time = end - start

(mycount,bits) = analyze(counts[0])
for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    
print("From QC:",counts)
print("Simulation elasped time:",qc_time)


In [ ]:
print("="*50)
print("Start run:")
start = time.time()        
iters = 1
counts = fire_ibmq(circuit,qc_shots,iters,False,False,backend_name="ibmq_16_melbourne")
end = time.time()
qc_time = end - start

(mycount,bits) = analyze(counts[0])
for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    
print("From QC:",counts)
print("Simulation elasped time:",qc_time)




In [ ]:
print("="*50)
print("Start run:")
start = time.time()        
iters = 1
counts = fire_ibmq(circuit,qc_shots,iters,False,False,backend_name="ibmq_essex")
end = time.time()
qc_time = end - start

(mycount,bits) = analyze(counts[0])
for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    
print("From QC:",counts)
print("Simulation elasped time:",qc_time)


print("="*50)
print("Start run:")
start = time.time()        
iters = 1
counts = fire_ibmq(circuit,qc_shots,iters,False,False,backend_name="ibmq_london")
end = time.time()
qc_time = end - start

(mycount,bits) = analyze(counts[0])
for b in range(bits):
    print (b,float(mycount[b])/qc_shots)

print("From QC:",counts)
print("Simulation elasped time:",qc_time)


|   Input   |    Quantum  |   Res  |  Time |
|:---------:|:-----------:|:------:|:-----:|
| [0.9,0.1] |  theoretic  | 18.00% | 1s    |
| [0.9,0.1] |  simulation | 18.16% | 5s    |
| [0.9,0.1] |    essex    | 21.88% | 105s  | 
| [0.9,0.1] |    london   | 19.92% | 92s   |
|
| [0.8,0.2] |  theoretic  | 32.00% | 5s    |
| [0.8,0.2] |  simulation | 30.27% | 5s    |
| [0.8,0.2] |    essex    | 35.94% | 124s  | 
| [0.8,0.2] |    london   | 28.71% | 341s  |
|
| [0.7,0.3] |  theoretic  | 42.00% | 1s    |
| [0.7,0.3] |  simulation | 41.99% | 1s    |
| [0.7,0.3] |    essex    | 40.82% | 45s   | 
| [0.7,0.3] |    london   | 35.74% | 18s   |
|
| [0.6,0.4] |  theoretic  | 48.00% | 1s    |
| [0.6,0.4] |  simulation | 47.75% | 1s    |
| [0.6,0.4] |    essex    | 43.26% | 22s   | 
| [0.6,0.4] |    london   | 43.65% | 44s   |
|
| [0.5,0.5] |  theoretic  | 50.00% | 1s    |
| [0.5,0.5] |  simulation | 49.02% | 1s    |
| [0.5,0.5] |    essex    | 44.34% | 164s  | 
| [0.5,0.5] |    london   | 48.05% | 35s   |



|       Input       |    Quantum  |   Res  |  Time |
|:-----------------:|:-----------:|:------:|:-----:|
| [0.6,0.4,0.3,0.1] |  theoretic  | 28.50% | 1s    |
| [0.6,0.4,0.3,0.1] |  simulation | 28.52% | 5s    |
| [0.6,0.4,0.3,0.1] |   melbourne | 40.92% | 253s  |

In [97]:
print(torch.tensor(1-2*(0.1+0.9-2*0.1*0.9)).acos())


tensor(2.2653)


In [ ]:

input_ori = [0.1,0.2,0.3,0.4]
input_ori = [0.4,0.2,0.7,0.0]
input_ori = [random.random(),random.random(),random.random(),random.random()]
# input_ori = [0.2250, 0.3050, 0.2250, 0.3050]
# input_ori = [0.4141, 0.2688, 0.2312, 0.1932]
# input_ori = [0.3924, 0.2127, 0.2247, 0.2099]
# input_ori = [0.4170, 0.2226, 0.2284, 0.1938]
# input_ori = [0.4072, 0.2137, 0.2262, 0.1990]
w1    = [ 0.0, 0.0, 0.0, 0.0]
w2    = [ 0.0, 0.0, 0.0, 1.0]
w3    = [ 0.0, 0.0, 1.0, 0.0]
w4    = [ 0.0, 0.0, 1.0, 1.0]
w5    = [ 0.0, 1.0, 0.0, 0.0]
w6    = [ 0.0, 1.0, 0.0, 1.0]
w7    = [ 0.0, 1.0, 1.0, 0.0]
w8    = [ 0.0, 1.0, 1.0, 1.0]
w9    = [ 1.0, 0.0, 0.0, 0.0]
w10    = [ 1.0, 0.0, 0.0, 1.0]
w11    = [ 1.0, 0.0, 1.0, 0.0]
w12    = [ 1.0, 0.0, 1.0, 1.0]
w13    = [ 1.0, 1.0, 0.0, 0.0]
w14    = [ 1.0, 1.0, 0.0, 1.0]
w15    = [ 1.0, 1.0, 1.0, 0.0]
w16    = [ 1.0, 1.0, 1.0, 1.0]


t_w1 = torch.tensor([w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16])*2-1

for i in range(100):
    
    input_ori = [random.random(),random.random(),random.random(),random.random()]    
    t_input = torch.tensor([input_ori])*2-1
    t_output = do_slp_via_th(t_input,t_w1).flatten()
    
    # input_ori = random.sample(list(t_output),4)
    
    t_out_mean = t_output.sum()/t_output.shape[0]
    
    print((t_output-t_out_mean).pow(2).sum()/t_output.shape[0],input_ori)
    